# Primary Notebook

## Purpose

The purpose of this notebook is the creation of the clusters.
Those (landscape) clusters are supposed to be areas similar in appearance;
To be more precise, these clusters share a common "FFT footprint" which will be elaborated on later.

## Other parts of this project

There are secondary notebooks, that might have to be run beforehand, to receive data which this primary notebook will work upon.

In [7]:
# What we might need eventually
# rasterio for geotiffs
# dask array for parallel computing of large arrays
# pyfftw for 2d fft 

## Overview of the classes and their interconnection
The subject of this notebook is to cluster different landscapes across multiple DEM-GeoTIFFs. 

### GeographicBounds
This is an object that saves West, South, East, North borders, as well as the projection, and maybe some other additional info.

### AugmentedDEM
This will be the "container" for all information regarding *one* specific DEM raster map.

### EmbeddingMap
This will be part of each AugmentedDEM. Here we will save the labels created by the clustering.




## How to make the GeoTIFF images usable by my algorithm:
It’s important to note, that the original DEM data is in arc seconds. To avoid skewed results it has to be compressed by cos(latitude) in width.  That way the metric distances (almost) resemble the distances pixel-wise

After that the images will be scaled to 1/4 (as of now), because processing becomes about 4^2 times faster.

Before processing, the data will have to be split up into quadratic tiles which are supposed to be equal in their length.

I guess we should combine the processes, namely selecting tiles of equal size and then reducing their size to a given size in pixels

In [8]:
# Imports

# Filesystem, JSON
import os
import json
from io import BytesIO # To be able to route the API response to a file-like object that rasterio can use

# Rasterio for handling GeoTIFFs
import rasterio

# Math may not be missing
import math
import numpy as np

# Regex for updating filenames
import re

# For earth related numbers
from pyproj import Geod
geod = Geod(ellps="WGS84")

In [9]:
# Settings for running this Notebook

internal_settings = {}

internal_settings["files"] = {}
internal_settings["files"]["dem_folder"] = "geotiffs" #Here the to-be-used-geotiffs are located

internal_settings["fft"] = {}
internal_settings["fft"]["tile_size"] = 1.75 #Average length and width of a tile that is processed individually

temporary_settings = {}
temporary_settings["augmented_dems"] = []

In [ ]:
# Define classes specific to this project 

# This is just a coordinate pair, have to decide whether we need it OPTIONAL
class GeographicPoint:
    def __init__(self, north, east):
        self.north = north
        self.east = east

    def __str__(self): # Make it printable
        infostring = f"Geographic Point at\n"
        infostring += f"North: {self.north:>7.2f}\n"
        infostring += f"East: {self.east:>8.2f}° \n\n"
        return infostring

# This class is supposed to store coordinates for tiles and maybe coordinate systems in the future
class GeographicBounds:
    def __init__(self, west, south, east, north):
        self.west = west
        self.north = north
        self.east = east
        self.south = south

    @property
    def height (self): # Height in kilometres
        return (abs(self.south - self.north) / 360.0) * math.pi * 2.0 * geod.b / 1000.0; 
   
    @property
    def width_north (self): # North width in kilometres
        return (abs(self.west - self.east) / 360.0) * math.pi * 2.0 * geod.a * math.cos( math.radians(self.north) ) / 1000.0 ; 
   
    @property
    def width_south (self): # North width in kilometres
        return (abs(self.west - self.east) / 360.0) * math.pi * 2.0 * geod.a * math.cos( math.radians(self.south) ) / 1000.0 ; 

    # Return information about the object (for to be used in a print statement for example)
    def __str__(self):
        infostring = "Infos about the GeographicBounds (all rounded):\n"
        infostring += f"-----\n"
        infostring += f"Geographic Bounds:\n"
        infostring += f"West: {self.west:>8.2f}° \n"
        infostring += f"South: {self.south:>7.2f}° \n"
        infostring += f"East: {self.east:>8.2f}° \n"
        infostring += f"North: {self.north:>7.2f}\n"
        infostring += f"-----\n"
        infostring += f"Northern Width: {self.width_north:.2f} km\n"
        infostring += f"Southern Width: {self.width_south:.2f} km\n"
        infostring += f"Height: {self.height:.2f} km\n\n"
        return infostring
    
    def as_list(self):
        return [self.west, self.south, self.east, self.north]

# This class will contain information about each geotiff that is used for the fft calculations
class AugmentedDEM:
    def __init__(self, dem_path):
        with rasterio.open(dem_path) as dem_file:
            self.geo_bounds = GeographicBounds(dem_file.bounds.left, 
                                               dem_file.bounds.bottom,
                                               dem_file.bounds.right,
                                               dem_file.bounds.top)
        self.dem_path = dem_path


    def split_into_tiles(self):
        self.tiles_y_count = self.geo_bounds.height / internal_settings["fft"]["tile_size"]
        self.tiles_y_relative_size = 1 / self.tiles_y_count
        self.tiles_y_rest = self.tiles_y_count % 1;
        self.tiles_y_relative_start = (self.tiles_y_rest / 2) / self.tiles_y_count
        self.tiles_y_relative_end = 1 - self.tiles_y_relative_start

        # intentional temporary variable
        tiles_y_center_first_row = np.array([self.tiles_y_relative_start / 2])
        tiles_y_center_last_row = np.array([1 - (self.tiles_y_relative_start / 2)])

        tiles_y_center_internal_rows = np.linspace(self.tiles_y_relative_start + (self.tiles_y_relative_size / 2),
                                                   self.tiles_y_relative_end - (self.tiles_y_relative_size / 2),
                                                   math.floor(self.tiles_y_count), endpoint=True)

        self.tiles_y_borders = np.linspace(self.tiles_y_relative_start, 
                                      self.tiles_y_relative_end, 
                                      math.ceil(self.tiles_y_count))
        
        self.tiles_y_centers = np.concatenate((tiles_y_center_first_row, tiles_y_center_internal_rows, tiles_y_center_last_row))
        print(self.tiles_y_centers)


    def tileize(self): # alternative implementation that gives back the borders and stuff in degrees
        # Height of a tile in degrees
        tile_height_degrees = 360 * internal_settings["fft"]["tile_size"] /  (math.pi * 2.0 * geod.b) 

        start_y = (self.geo_bounds.height % tile_height_degrees) / 2
        number_tiles_y = self.geo_bounds.height // tile_height_degrees

        borders_y = np.linspace(start_y, height - start_y, number_tiles_y + 1 )

        



In [ ]:
# Just a Testing Cell Block

height = 90
tile = 35
start_y = (height % tile) / 2
number_tiles_y = int( height // tile )


borders_y = np.linspace(start_y, height - start_y, number_tiles_y + 1 )

print(borders)

[10. 45. 80.]


In [28]:
# Put all the DEM maps from the dem_folder into the dem_files
with os.scandir(internal_settings["files"]["dem_folder"]) as dirlist:
    temporary_settings["augmented_dems"] =[AugmentedDEM(d.path) for d in dirlist if re.search(r"\.tif",d.name)]



In [22]:
# Testing
for d in temporary_settings["augmented_dems"]:
    print(d.geo_bounds)

temporary_settings["augmented_dems"][1].split_into_tiles()

Infos about the GeographicBounds (all rounded):
-----
Geographic Bounds:
West:     5.17° 
South:   43.05° 
East:     6.42° 
North:   43.94
-----
Northern Width: 100.20 km
Southern Width: 101.68 km
Height: 98.74 km


Infos about the GeographicBounds (all rounded):
-----
Geographic Bounds:
West:     6.50° 
South:   46.93° 
East:     8.60° 
North:   48.02
-----
Northern Width: 156.36 km
Southern Width: 159.64 km
Height: 120.93 km


Infos about the GeographicBounds (all rounded):
-----
Geographic Bounds:
West:    -7.14° 
South:   55.20° 
East:    -3.73° 
North:   56.80
-----
Northern Width: 207.85 km
Southern Width: 216.64 km
Height: 177.51 km


[3.75048936e-04 7.98560370e-03 2.24566154e-02 3.69276270e-02
 5.13986387e-02 6.58696503e-02 8.03406620e-02 9.48116736e-02
 1.09282685e-01 1.23753697e-01 1.38224709e-01 1.52695720e-01
 1.67166732e-01 1.81637744e-01 1.96108755e-01 2.10579767e-01
 2.25050779e-01 2.39521790e-01 2.53992802e-01 2.68463814e-01
 2.82934825e-01 2.97405837e-01 3.11876848e-01

## Code to be refactored and integrated: